<a href="https://www.kaggle.com/code/klyushnik/my-work?scriptVersionId=168508462" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.isotonic import IsotonicRegression
import random
import os
from copy import deepcopy
from functools import partial
from itertools import combinations

# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import seaborn as sns
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor

import optuna

import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool
from h2o.automl import H2OAutoML

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/playground-series-s3e19/sample_submission.csv
/kaggle/input/playground-series-s3e19/train.csv
/kaggle/input/playground-series-s3e19/test.csv


# 1.Load data

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv")

In [3]:
print('Shape train =', train.shape)
print('Shape test =', test.shape)

Shape train = (136950, 6)
Shape test = (27375, 5)


# 2.Auto viz

In [4]:
!pip install AutoViz
!pip install xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 9.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of panel to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
%matplotlib inline
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()

Imported v0.1.806. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


In [6]:
dftc = AV.AutoViz('/kaggle/input/playground-series-s3e19/train.csv')

Shape of your Data Set loaded: (136950, 6)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    Number of Numeric Columns =  0
    Number of Integer-Categorical Columns =  1
    Number of String-Categorical Columns =  3
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  0
    Number of Discrete String Columns =  0
    Number of NLP String Columns =  1
    Number of Date Time Columns =  0
    Number of ID Columns =  1
    Number of Columns to Delete =  0
    6 Predictors classified...
        1 variable(s) removed since they were ID or low-information variables
        List of variables removed: ['id']
To fix these data quality issues in the dataset, import FixDQ

,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
id,int64,0.000000,100,0.000000,136949.000000,Possible ID column: drop before modeling step.
date,object,0.000000,1,,,No issue
country,object,0.000000,0,,,No issue
store,object,0.000000,0,,,No issue
product,object,0.000000,0,,,No issue
num_sold,int64,0.000000,0,2.000000,1380.000000,Column has 18906 outliers greater than upper bound (391.00) or lower than lower bound(-161.00). Cap them or remove them.


Could not draw Distribution Plot
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package movie_rev

# 3.Data preparation

In [7]:
train = pd.get_dummies (train, columns=['country','store', 'product'], drop_first= True )
test = pd.get_dummies (test, columns=['country','store', 'product'], drop_first= True )

print('Shape train =', train.shape)
print("Shape test =", test.shape)

Shape train = (136950, 13)
Shape test = (27375, 12)


In [8]:
import calendar
import datetime
from learntools.time_series.style import *

In [9]:
train.date = pd.to_datetime(train.date)
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.dayofweek
train['dayofyear'] = train['date'].dt.dayofyear
train['week'] = train['date'].dt.isocalendar().week
train['quarter'] = train['date'].dt.quarter
train['day_of_week'] = train['date'].dt.day_name()
train.head()

,id,date,num_sold,country_Canada,country_Estonia,country_Japan,country_Spain,store_Kaggle Learn,store_Kaggle Store,product_Using LLMs to Train More LLMs,product_Using LLMs to Win Friends and Influence People,product_Using LLMs to Win More Kaggle Competitions,product_Using LLMs to Write Better,year,month,day,weekday,dayofyear,week,quarter,day_of_week
0,0,2017-01-01,63,0,0,0,0,1,0,0,0,0,0,2017,1,1,6,1,52,1,Sunday
1,1,2017-01-01,66,0,0,0,0,1,0,1,0,0,0,2017,1,1,6,1,52,1,Sunday
2,2,2017-01-01,9,0,0,0,0,1,0,0,1,0,0,2017,1,1,6,1,52,1,Sunday
3,3,2017-01-01,59,0,0,0,0,1,0,0,0,1,0,2017,1,1,6,1,52,1,Sunday
4,4,2017-01-01,49,0,0,0,0,1,0,0,0,0,1,2017,1,1,6,1,52,1,Sunday


In [10]:
test.date = pd.to_datetime(test.date)
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.dayofweek
test['dayofyear'] = test['date'].dt.dayofyear
test['week'] = test['date'].dt.isocalendar().week
test['quarter'] = test['date'].dt.quarter
test['day_of_week'] = test['date'].dt.day_name()
test.head()

,id,date,country_Canada,country_Estonia,country_Japan,country_Spain,store_Kaggle Learn,store_Kaggle Store,product_Using LLMs to Train More LLMs,product_Using LLMs to Win Friends and Influence People,product_Using LLMs to Win More Kaggle Competitions,product_Using LLMs to Write Better,year,month,day,weekday,dayofyear,week,quarter,day_of_week
0,136950,2022-01-01,0,0,0,0,1,0,0,0,0,0,2022,1,1,5,1,52,1,Saturday
1,136951,2022-01-01,0,0,0,0,1,0,1,0,0,0,2022,1,1,5,1,52,1,Saturday
2,136952,2022-01-01,0,0,0,0,1,0,0,1,0,0,2022,1,1,5,1,52,1,Saturday
3,136953,2022-01-01,0,0,0,0,1,0,0,0,1,0,2022,1,1,5,1,52,1,Saturday
4,136954,2022-01-01,0,0,0,0,1,0,0,0,0,1,2022,1,1,5,1,52,1,Saturday


In [11]:
print('Shape train =', train.shape)
print("Shape test =", test.shape)

Shape train = (136950, 21)
Shape test = (27375, 20)


In [12]:
from sklearn.preprocessing import OrdinalEncoder
List_a = ['day_of_week']

lable = OrdinalEncoder()
for i in List_a:
    train.loc[:, i] = lable.fit_transform(train[[i]])
    test.loc[:, i] = lable.fit_transform(test[[i]])

In [13]:
train = train.drop(columns=['id', 'date'], axis =1)
test = test.drop(columns=['id', 'date'], axis =1)

print('Shape train =', train.shape)
print("Shape test =", test.shape)

Shape train = (136950, 19)
Shape test = (27375, 18)


In [14]:
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn import preprocessing
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [15]:
list_b = [
     'country_Canada', 'country_Estonia', 'country_Japan',
       'country_Spain', 'store_Kaggle Learn', 'store_Kaggle Store',
       'product_Using LLMs to Train More LLMs',
       'product_Using LLMs to Win Friends and Influence People',
       'product_Using LLMs to Win More Kaggle Competitions',
       'product_Using LLMs to Write Better']

for i in list_b:
    train[i+' +log'] = (train[i]+1).transform(np.log)
    test[i+' +log'] =(test[i]+1).transform(np.log)
######################################################
for i in list_b:
    train[i+'+log1'] = (train[i]+1).transform(np.log1p)
    test[i+'+log1'] =(test[i]+1).transform(np.log1p)
######################################################
transform = PowerTransformer(method='yeo-johnson')
for i in list_b:
    train[i+'+y_j'] = transform.fit_transform(train[[i]])
    test[i+'+y_j'] = transform.fit_transform(test[[i]])
######################################################
transform = QuantileTransformer(n_quantiles=10, random_state=0)
for i in list_b:
    train[i+'+q_t'] = transform.fit_transform(train[[i]])
    test[i+'+q_t'] = transform.fit_transform(test[[i]])
######################################################
for i in list_b:
    train[i+'+sqrt'] = (train[i]+1).transform(np.sqrt)
    test[i+'+sqrt'] =(test[i]+1).transform(np.sqrt)
######################################################
maxabs = MaxAbsScaler()
for i in list_b:
    train[i+'+abs'] = maxabs.fit_transform(train[[i]])
    test[i+'+abs'] =maxabs.fit_transform(test[[i]])
######################################################
minmax = MinMaxScaler()
for i in list_b:
    train[i+'+minmax'] = minmax.fit_transform(train[[i]])
    test[i+'+minmax'] = minmax.fit_transform(test[[i]])
######################################################
rob = RobustScaler()
for i in list_b:
    train[i+'+rob'] = rob.fit_transform(train[[i]])
    test[i+'+rob'] = rob.fit_transform(test[[i]])
######################################################
norm = preprocessing.Normalizer()
for i in list_b:
    train[i+'+norm'] = norm.fit_transform(train[[i]])
    test[i+'+norm'] = norm.fit_transform(test[[i]])
######################################################
from sklearn.preprocessing import SplineTransformer
spline = SplineTransformer()

print('Shape train =', train.shape)
print("Shape test =", test.shape)

Shape train = (136950, 109)
Shape test = (27375, 108)


# 4.Polynomial Features labeled

In [16]:
def PolynomialFeatures_labeled(input_df,power):
   
    poly = preprocessing.PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_

    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s+%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + "x" + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df
train_d = train.drop(columns=['num_sold'], axis =1)
output_df = PolynomialFeatures_labeled(train_d.iloc[:,:50],2)
output_df_t = PolynomialFeatures_labeled(test.iloc[:,:50],2)

In [17]:
train =  pd.concat([train.reset_index(drop=True), output_df], axis = 1)
test = pd.concat([test.reset_index(drop=True), output_df_t], axis = 1)

print('Shape train =', train.shape)
print('Shape test =', test.shape)

Shape train = (136950, 1435)
Shape test = (27375, 1434)


# 5.Removing infinite numbers and PCA

In [18]:
train = train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
test = test.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

print('Shape train =', train.shape)
print('Shape test =', test.shape)

Shape train = (136950, 1435)
Shape test = (27375, 1434)


In [19]:
train = train.astype(np.float)
test = test.astype(np.float)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Columns: 1435 entries, num_sold to country_Estonia+q_t+2
dtypes: float64(1435)
memory usage: 1.5 GB


In [20]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents_train = pca.fit_transform(train)
principalDf_train = pd.DataFrame(data = principalComponents_train, 
                                         columns = ['principal component 1',
                                                    'principal component 2'])
principalComponents_test = pca.fit_transform(test)
principalDf_test = pd.DataFrame(data = principalComponents_test, 
                                        columns = ['principal component 1',
                                                   'principal component 2'])
train = pd.concat([train, principalDf_train], axis=1)
test = pd.concat([test, principalDf_test], axis=1)

print('Shape train =', train.shape)
print('Shape test =', test.shape)

Shape train = (136950, 1437)
Shape test = (27375, 1436)


# 6.StandardScaler and feature_selection

In [21]:
from sklearn.feature_selection import VarianceThreshold
def variance_threshold(df,th):
    var_thres=VarianceThreshold(threshold=th)
    var_thres.fit(df)
    new_cols = var_thres.get_support()
    return df.iloc[:,new_cols]

In [22]:
y = train['num_sold']
X = variance_threshold(train.drop(['num_sold'],axis=1),0.4)
list_name = (X.columns)
X_total = test[list_name]

print('Shape train =', X.shape)
print('Shape test =', X_total.shape)

Shape train = (136950, 565)
Shape test = (27375, 565)


# 7.Parametrs

In [23]:
early_stopping_rounds=200
n_splits=5
n_estimators = 3000


xgb_params = {
    'n_estimators': n_estimators,
    'learning_rate': 0.171655957085321,
    'booster': 'gbtree',
    'lambda': 0.0856890877950814,
    'alpha': 0.000223555755009136,
    'subsample': 0.817828998442447,
    'colsample_bytree': 0.247566473896556,
    'max_depth': 9,
    'min_child_weight': 6,
    'eta': 0.0000146950106167549,
    'gamma': 0.0315067984172879,
    'grow_policy': 'depthwise',
    'n_jobs': -1,
    'objective': 'reg:squarederror', 
    'eval_metric': 'mape',
    'verbosity': 0}

lgb_params = {
   'n_estimators': n_estimators,
   'learning_rate': 0.10888790969844592,
    'min_child_samples': 20,
    'num_leaves': 90}

cb_params = {
    'iterations': n_estimators,
    'depth': 6,
    'learning_rate': 0.00454306521731278,
    'l2_leaf_reg': 0.113774158297261,
    'random_strength': 0.0179641854849499,
    'od_type': 'IncToDec',
    'bootstrap_type': 'Bayesian',
    'grow_policy': 'Lossguide',
    'verbose': False}

cat_params_2 = {
    
    'iterations': n_estimators, 
    'learning_rate': 0.1252297003236841, 
    'depth': 9, 
    'l2_leaf_reg': 3.783310969240252, 
    'subsample': 0.5758950122641308, 
    'colsample_bylevel': 0.1360009677406299,
    'random_state': 42,
    'verbose': False
}

lgb_params_2 = {
    'n_estimators': n_estimators,
    'num_leaves': 123,
    'max_depth': 15,
    'learning_rate': 0.09994734544846036,
    'subsample': 0.7677561538969853,
    'colsample_bytree': 0.8165353823844163,
    'reg_lambda': 7.771108927781672e-06
}

# 8.Train regressor

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import FunctionTransformer
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor

In [25]:
FOLDs = KFold(n_splits=n_splits, shuffle=True,random_state=42)

oof_cat = np.zeros(len(X))
predictions_cat = np.zeros(len(X_total))
oof_xgb = np.zeros(len(X))
predictions_xgb = np.zeros(len(X_total))
oof_lgb = np.zeros(len(X))
predictions_lgb = np.zeros(len(X_total))
oof_hgb = np.zeros(len(X))
predictions_hgb = np.zeros(len(X_total))
oof_cat_2 = np.zeros(len(X))
predictions_cat_2 = np.zeros(len(X_total))
oof_lgb_2 = np.zeros(len(X))
predictions_lgb_2 = np.zeros(len(X_total))

for fold_, (trn_idx, val_idx) in enumerate(FOLDs.split(X,y)):
    X.iloc[trn_idx], y.iloc[trn_idx]
    X.iloc[val_idx], y.iloc[val_idx]

#cat#######################################################################
    cat_model = Pipeline([('scaler', StandardScaler()),
                          ('cat',CatBoostRegressor(**cb_params,
                                                   early_stopping_rounds=early_stopping_rounds))])
    cat_model.fit(X.iloc[trn_idx],y.iloc[trn_idx])
    oof_cat[val_idx] = cat_model.predict(X.iloc[val_idx])

    predictions_cat += cat_model.predict(X_total) / FOLDs.n_splits
    
    cat_score = mean_absolute_error(oof_cat, y)

    print('Fold', fold_, '-- CatBoostRegressor oof MAE is ---',cat_score)
    
#xgb#######################################################################    
    
    xgb_model = Pipeline([('scaler', StandardScaler()),
                          ('xgb', xgb.XGBRegressor(**xgb_params))])
    

    xgb_model.fit(X.iloc[trn_idx],y.iloc[trn_idx])
   
    oof_xgb[val_idx] = xgb_model.predict(X.iloc[val_idx])

    predictions_xgb += xgb_model.predict(X_total)/ FOLDs.n_splits
    
    xgb_score = mean_absolute_error(oof_xgb, y)

    print('Fold', fold_, '-- XGBRegressor oof MAE is ---',xgb_score)

#lgb#######################################################################
    pipeline_temp = Pipeline([('scaler', StandardScaler())])
    X_trans = pipeline_temp.fit_transform(X.iloc[val_idx],y.iloc[val_idx])
    eval_set = [(X_trans, y.iloc[val_idx])]
    
    lgb_model = Pipeline([('scaler', StandardScaler()),
                          ('lgb',lgb.LGBMRegressor(**lgb_params))])
    
    lgb_model.fit(X.iloc[trn_idx],y.iloc[trn_idx])
    oof_lgb[val_idx] = lgb_model.predict(X.iloc[val_idx])


    predictions_lgb += lgb_model.predict(X_total) / FOLDs.n_splits
    
    lgb_score = mean_absolute_error(oof_lgb, y)
    
    print('Fold', fold_, '-- LGBMRegressor oof MAE is ---', lgb_score)

#hbr#######################################################################
    hgb_model = Pipeline([('scaler', StandardScaler()),
                          ('hgb',HistGradientBoostingRegressor(max_iter=n_estimators,
                                                               learning_rate=0.01,
                                                               loss="absolute_error",
                                                               n_iter_no_change=50,
                                                               random_state=42))])
    
    hgb_model.fit(X.iloc[trn_idx],y.iloc[trn_idx])
    oof_hgb[val_idx] = hgb_model.predict(X.iloc[val_idx])
    predictions_hgb += hgb_model.predict(X_total) / FOLDs.n_splits
    hgb_score = mean_absolute_error(oof_hgb, y)
    
    print('Fold', fold_, '-- HistGradientBoostingRegressor oof MAE is ---', hgb_score)
#cat#######################################################################
    cat_model_1 = Pipeline([('scaler', StandardScaler()),
                          ('cat',CatBoostRegressor(**cat_params_2,
                                                   early_stopping_rounds=early_stopping_rounds))])
    cat_model_1.fit(X.iloc[trn_idx],y.iloc[trn_idx])
    oof_cat_2[val_idx] = cat_model_1.predict(X.iloc[val_idx])

    predictions_cat_2 += cat_model.predict(X_total) / FOLDs.n_splits
    
    cat_score_2 = mean_absolute_error(oof_cat_2, y)

    print('Fold', fold_, '-- CatBoostRegressor_2 oof MAE is ---',cat_score_2)
#lgb#######################################################################
    pipeline_temp = Pipeline([('scaler', StandardScaler())])
    X_trans = pipeline_temp.fit_transform(X.iloc[val_idx],y.iloc[val_idx])
    eval_set = [(X_trans, y.iloc[val_idx])]
    
    lgb_model_2 = Pipeline([('scaler', StandardScaler()),
                          ('lgb',lgb.LGBMRegressor(**lgb_params_2))])
    
    lgb_model_2.fit(X.iloc[trn_idx],y.iloc[trn_idx])
    oof_lgb_2[val_idx] = lgb_model_2.predict(X.iloc[val_idx])


    predictions_lgb_2 += lgb_model_2.predict(X_total) / FOLDs.n_splits
    
    lgb_score_2 = mean_absolute_error(oof_lgb_2, y)
    
    print('Fold', fold_, '-- LGBMRegressor_2 oof MAE is ---', lgb_score_2)

Fold 0 -- CatBoostRegressor oof MAE is --- 134.21123934678883
Fold 0 -- XGBRegressor oof MAE is --- 134.3070670033779
Fold 0 -- LGBMRegressor oof MAE is --- 134.0691714650535
Fold 0 -- HistGradientBoostingRegressor oof MAE is --- 134.0965709134045
Fold 0 -- CatBoostRegressor_2 oof MAE is --- 133.95689018838647
Fold 0 -- LGBMRegressor_2 oof MAE is --- 134.09862639001372
Fold 1 -- CatBoostRegressor oof MAE is --- 102.82553736419928
Fold 1 -- XGBRegressor oof MAE is --- 103.04457314625492
Fold 1 -- LGBMRegressor oof MAE is --- 102.56849316685175
Fold 1 -- HistGradientBoostingRegressor oof MAE is --- 102.64102088155313
Fold 1 -- CatBoostRegressor_2 oof MAE is --- 102.34592037458916
Fold 1 -- LGBMRegressor_2 oof MAE is --- 102.62818307994075
Fold 2 -- CatBoostRegressor oof MAE is --- 71.31074285159755
Fold 2 -- XGBRegressor oof MAE is --- 71.62808434390112
Fold 2 -- LGBMRegressor oof MAE is --- 70.92709998436436
Fold 2 -- HistGradientBoostingRegressor oof MAE is --- 71.03325955662861
Fold 2

# 9.Blend

In [26]:
train_df = pd.DataFrame({#'1': oof_cat,
                         #'2': oof_xgb,
                         '3': oof_lgb,
                         '4': oof_hgb,
                         '5': oof_cat_2,
                         '6': oof_lgb_2
                         })
y = train['num_sold']

test_df = pd.DataFrame({#'1': predictions_cat,  
                        #'2': predictions_xgb, 
                        '3': predictions_lgb, 
                        '4': predictions_hgb,
                        '5': predictions_cat_2,
                        '6': predictions_lgb_2
                        })
for i in train_df.columns:
    train_df[i+' +log1'] = (train_df[i]+1).transform(np.log1p)
    test_df[i+' +log1'] =(test_df[i]+1).transform(np.log1p)
    train_df[i+' +log'] = (train_df[i]+1).transform(np.log)
    test_df[i+' +log'] =(test_df[i]+1).transform(np.log)

FOLDs = KFold(n_splits=n_splits, shuffle=True,random_state=42)

oof_blend = np.zeros(len(train_df))
predictions_blend = np.zeros(len(test_df))
for fold_, (trn_idx, val_idx) in enumerate(FOLDs.split(train_df,y)):
    train_df.iloc[trn_idx], y.iloc[trn_idx]
    train_df.iloc[val_idx], y.iloc[val_idx]

    blend = lgb.LGBMRegressor(objective = 'mae',
                              n_estimators = 250,
                              max_depth = 10,
                              learning_rate = 0.01)
    blend.fit(train_df.iloc[trn_idx],y.iloc[trn_idx])
    
    oof_blend[val_idx] = blend.predict(train_df.iloc[val_idx])
    predictions_blend += blend.predict(test_df)/FOLDs.n_splits
    blend_score = mean_absolute_error(oof_blend, y)
    
    print('Fold', fold_, '-- Blend oof MAE is ---',blend_score)

Fold 0 -- Blend oof MAE is --- 135.41480333980488
Fold 1 -- Blend oof MAE is --- 105.2735045957282
Fold 2 -- Blend oof MAE is --- 74.98247051000423
Fold 3 -- Blend oof MAE is --- 45.2385647106402
Fold 4 -- Blend oof MAE is --- 14.871693052920211


In [27]:
print("Blend: {:<8.5f}".format(mean_absolute_error(oof_blend, y)))

Blend: 14.87169


# 11.Submit

In [28]:
sub = pd.read_csv('/kaggle/input/playground-series-s3e19/sample_submission.csv')#/kaggle/input/playground-series-s3e19/sample_submission.csv
sub['num_sold'] = predictions_blend 
sub.to_csv('submission.csv', index=False)
sub

,id,num_sold
0,136950,44.485392
1,136951,44.321477
2,136952,25.557739
3,136953,41.920214
4,136954,33.996438
...,...,...
27370,164320,561.219621
27371,164321,561.219621
27372,164322,114.630611
27373,164323,561.219621
